In [1]:
import tensorflow as tf
import matplotlib.pyplot as plt
import numpy as np
import tensorflow_datasets as tfds

In [2]:
import urllib3
urllib3.disable_warnings()

#tfds.disable_progress_bar()   # 이 주석을 풀면 데이터셋 다운로드과정의 프로그레스바가 나타나지 않습니다.

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    shuffle_files=True,
    with_info=True,
)

In [3]:
def normalize_and_resize_img(image, label):
    """Normalizes images: `uint8` -> `float32`."""
    # image = tf.image.resize(image, [32, 32])
    return tf.cast(image, tf.float32) / 255., label

In [4]:
def apply_normalize_on_dataset(ds, is_test=False, batch_size=16):
    ds = ds.map(
        normalize_and_resize_img, 
        num_parallel_calls=1
    )
    ds = ds.batch(batch_size)
    if not is_test:
        ds = ds.repeat()
        ds = ds.shuffle(200)
    ds = ds.prefetch(tf.data.experimental.AUTOTUNE)
    return ds

In [5]:
class ConvBlock(tf.keras.Model):
    def __init__(self, channel, kernel_size, strides = 1):
        super(ConvBlock,self).__init__()
        self.conv1 = tf.keras.layers.Conv2D(channel, kernel_size=kernel_size, strides=strides, padding='same')
        self.bn1 = tf.keras.layers.BatchNormalization()
        
    def call(self, input_tensor, training = False):
        x = self.conv1(input_tensor)
        x = self.bn1(x, training = training)
        return x

In [6]:
class ResBlock(tf.keras.Model):
    def __init__(self, channel,down_sample = False):
        super(ResBlock,self).__init__()
        self.down_sample = down_sample
        
        if self.down_sample:
            self.conv_block1 = ConvBlock(channel, kernel_size=1, strides=2)
        else:
            self.conv_block1 = ConvBlock(channel, kernel_size=1, strides=1)
            
        self.conv_block2 = ConvBlock(channel,kernel_size=3, strides=1)     
        self.conv_block3 = ConvBlock(channel*4, kernel_size=1, strides=1)
        
        if self.down_sample:
            self.downsample_conv = tf.keras.layers.Conv2D(channel*4, kernel_size=1, strides=2, padding='same')
            self.downsample_bn = tf.keras.layers.BatchNormalization()
    
    def call(self, input_tensor, training = False):
        x = self.conv_block1(input_tensor)
        x = tf.nn.relu(x)
        x = self.conv_block2(x)
        x = tf.nn.relu(x)
        x = self.conv_block3(x)
        
        if self.down_sample:
            input_tensor = self.downsample_conv(input_tensor)
            input_tensor = self.downsample_bn(input_tensor, training=training)

        x = x + input_tensor
        x = tf.nn.relu(x)
        return x

In [7]:
class ResNet50(tf.keras.Model):
    def __init__(self):
        super(ResNet50,self).__init__()
        
        self.conv = tf.keras.layers.Conv2D(64, kernel_size=7, strides=2, padding='same')
        self.bn = tf.keras.layers.BatchNormalization()     
        self.maxpool = tf.keras.layers.MaxPool2D(pool_size=2)
               
        self.conv1  = tf.keras.Sequential()
        self.conv2  = tf.keras.Sequential()
        self.conv3  = tf.keras.Sequential()
        self.conv4  = tf.keras.Sequential()
        
        for i in range(3):
            if i==0:
                self.conv1.add(ResBlock(64, down_sample = True))
            else:
                self.conv1.add(ResBlock(64))

        for i in range(4):
            if i==0:
                self.conv2.add(ResBlock(128, down_sample = True))
            else:
                self.conv2.add(ResBlock(128))

        for i in range(6):
            if i==0:
                self.conv3.add(ResBlock(256, down_sample = True))
            else:
                self.conv3.add(ResBlock(256))

        for i in range(3):
            if i==0:
                self.conv4.add(ResBlock(512, down_sample = True))
            else:
                self.conv4.add(ResBlock(512))

        self.gap = tf.keras.layers.GlobalAveragePooling2D()
#         self.fc1 = tf.keras.layers.Dense(4096, activation='relu')
#         self.fc2 = tf.keras.layers.Dense(4096, activation='relu')
        self.fc3 = tf.keras.layers.Dense(10, activation='softmax')        
        
    def call(self, input_tensor, training = False):
        x = self.conv(input_tensor)
        x = self.bn(x)
        x = tf.nn.relu(x)
        x = self.maxpool(x)

        x = self.conv1(x)
        x = self.conv2(x)
        x = self.conv3(x)
        x = self.conv4(x)
        
        x = self.gap(x)
#         x = self.fc1(x)
#         x = self.fc2(x)
        x = self.fc3(x)
        return x

In [8]:
BATCH_SIZE = 256
EPOCH = 1

(ds_train, ds_test), ds_info = tfds.load(
    'cifar10',
    split=['train', 'test'],
    as_supervised=True,
    shuffle_files=True,
    with_info=True,
)
ds_train = apply_normalize_on_dataset(ds_train, batch_size=BATCH_SIZE)
ds_test = apply_normalize_on_dataset(ds_test, batch_size=BATCH_SIZE)

In [9]:
resnet50 = ResNet50()

resnet50.compile(
    loss='sparse_categorical_crossentropy',
    optimizer=tf.keras.optimizers.SGD(lr=0.01, clipnorm=1.),
    metrics=['accuracy'],
)

history_resnet50 = resnet50.fit(
    ds_train,
    steps_per_epoch=int(ds_info.splits['train'].num_examples/BATCH_SIZE),
    validation_steps=int(ds_info.splits['test'].num_examples/BATCH_SIZE),
    epochs=EPOCH,
    validation_data=ds_test,
    verbose=1
)

195/195 [==============================] - 14s 70ms/step - loss: 2.8360 - accuracy: 0.1319 - val_loss: 2.4727 - val_accuracy: 0.1093


In [10]:
resnet50.summary()

Model: "res_net50"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              multiple                  9472      
_________________________________________________________________
batch_normalization (BatchNo multiple                  256       
_________________________________________________________________
max_pooling2d (MaxPooling2D) multiple                  0         
_________________________________________________________________
sequential (Sequential)      (None, 4, 4, 256)         220032    
_________________________________________________________________
sequential_1 (Sequential)    (None, 2, 2, 512)         1230336   
_________________________________________________________________
sequential_2 (Sequential)    (None, 1, 1, 1024)        7129088   
_________________________________________________________________
sequential_3 (Sequential)    (None, 1, 1, 2048)        14

In [11]:
model = tf.keras.applications.ResNet50(include_top=False, weights = 'imagenet',classes=10, input_shape=(32,32,3))

In [12]:
model.summary()

Model: "resnet50"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 32, 32, 3)]  0                                            
__________________________________________________________________________________________________
conv1_pad (ZeroPadding2D)       (None, 38, 38, 3)    0           input_1[0][0]                    
__________________________________________________________________________________________________
conv1_conv (Conv2D)             (None, 16, 16, 64)   9472        conv1_pad[0][0]                  
__________________________________________________________________________________________________
conv1_bn (BatchNormalization)   (None, 16, 16, 64)   256         conv1_conv[0][0]                 
___________________________________________________________________________________________